In [93]:
# import libraries
import pandas as pd     # library for data analysis and manipulation tools
import numpy as np      # library for scientific computing and data manipulation
import pyodbc           # library for connecting to SQL Server
import lasio           # library for reading and writing Log ASCII Standard (LAS) files
import re              # library for regular expression matching operations
import os              # library for interacting with operating system

In [94]:
# Establish a connection to the Access database
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\Users\brand\OneDrive - Universidad Central del Ecuador\Tesis Información\AC_SACHA_AGO_2023_U_T.mdb;')

# Define your SQL queries with just columns we need
sql_query_maestra = 'SELECT WELL_LEGAL_NAME, WELLBORE_ID, COMPLETION_COORDINATE_X, COMPLETION_COORDINATE_Y, COMPLETION_LEGAL_NAME, RSVR_NAME, FIELD FROM MAESTRA'
# sql_query_sc_all = 'SELECT * FROM SC'
sql_query_sc = 'SELECT COMPLETION_LEGAL_NAME, FECHA_COMPLETACION, RESERVORIO FROM SC'
sql_mensual = 'SELECT * FROM MENSUAL'
sql_diaria = 'SELECT * FROM DIARIA'
sql_iny_diaria = 'SELECT * FROM INY_DIARIA'
sql_production_test = 'SELECT * FROM PRUEBA_POZO'

In [95]:
# Execute the queries and fetch the data into a DataFrames
df_maestra = pd.read_sql(sql_query_maestra, conn)
df_sc = pd.read_sql(sql_query_sc, conn)
df_mensual = pd.read_sql(sql_mensual, conn)
df_diaria = pd.read_sql(sql_diaria, conn)
df_iny_diaria = pd.read_sql(sql_iny_diaria, conn)
df_production_test = pd.read_sql(sql_production_test, conn)

# Close the database connection
conn.close()

C:\Users\brand\AppData\Local\Temp\ipykernel_19912\1705036465.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_maestra = pd.read_sql(sql_query_maestra, conn)
C:\Users\brand\AppData\Local\Temp\ipykernel_19912\1705036465.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sc = pd.read_sql(sql_query_sc, conn)
C:\Users\brand\AppData\Local\Temp\ipykernel_19912\1705036465.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mensual = pd.read_sql(sql_mensual, conn)


C:\Users\brand\AppData\Local\Temp\ipykernel_19912\1705036465.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_diaria = pd.read_sql(sql_diaria, conn)
C:\Users\brand\AppData\Local\Temp\ipykernel_19912\1705036465.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_iny_diaria = pd.read_sql(sql_iny_diaria, conn)
C:\Users\brand\AppData\Local\Temp\ipykernel_19912\1705036465.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_production_test = pd.read_sql(sql_production_test, conn)


# POZOS

In [96]:
df_maestra.head()

,WELL_LEGAL_NAME,WELLBORE_ID,COMPLETION_COORDINATE_X,COMPLETION_COORDINATE_Y,COMPLETION_LEGAL_NAME,RSVR_NAME,FIELD
0,SCH-002B,S-2B,289935.8200,9.955795e+06,SCH-002BTI,LOWER T,SCHS
1,SCH-002B,S-2B,289935.8200,9.955795e+06,SCH-002BUI,LOWER U,SCHS
2,SCH-002,S-2,289933.0700,9.955781e+06,SCH-002TI,LOWER T,SCHS
3,SCH-002,S-2,289933.0700,9.955781e+06,SCH-002UI,LOWER U,SCHS
4,SCH-003,S-3,295091.5183,9.968342e+06,SCH-003TI,LOWER T,SCHN1


In [97]:
df_sc.head()

,COMPLETION_LEGAL_NAME,FECHA_COMPLETACION,RESERVORIO
0,SCH-005UI,2016-06-24,UI
1,SCH-006TI,2016-06-24,TI
2,SCH-006UI,2016-06-24,UI
3,SCH-007T,2016-06-24,TI
4,SCH-007U,2016-06-24,UI


In [98]:
# determine years in the dataset
df_sc['FECHA_COMPLETACION'].dt.year.unique()

array([2016., 2017., 2019., 2018., 2020., 2022., 2023., 2021.,   nan])

In [99]:
# Marge the DataFrames
df_wells_merge = df_maestra.merge(df_sc, on='COMPLETION_LEGAL_NAME', how='inner')

# change the name of the columns to NOMBRE_COMPLETO	X	Y   FCOMP   CAMPO
df_wells_merge.rename(columns={'WELLBORE_ID':'NOMBRE_COMPLETO','COMPLETION_COORDINATE_X':'X',
                        'COMPLETION_COORDINATE_Y':'Y', 'FECHA_COMPLETACION':'FCOMP', 'FIELD':'CAMPO'}, inplace=True)

#TODO: check if there are duplicated wells have to be removed
df_wells_merge.drop_duplicates(subset=['NOMBRE_COMPLETO'], inplace=True)

# drop columns that are not needed
df_wells = df_wells_merge.drop(['WELL_LEGAL_NAME', 'RESERVORIO', 'RSVR_NAME', 'COMPLETION_LEGAL_NAME'], axis=1)

df_wells.head().round(3)

,NOMBRE_COMPLETO,X,Y,CAMPO,FCOMP
0,S-2B,289935.820,9955795.320,SCHS,2016-06-24
2,S-2,289933.070,9955780.980,SCHS,2016-06-24
4,S-3,295091.518,9968341.653,SCHN1,2016-06-24
6,S-5,295027.198,9962203.243,SCHN1,2016-06-24
10,S-6,293967.658,9968829.463,SCHN1,2016-06-24


# PRODUCCIÓN

## Mensual

In [100]:
# Order the DataFrame by the well name and the date
df_mensual.sort_values(by=['COMP_S_NAME', 'PROD_DT'], inplace=True)
df_mensual.reset_index(drop=True, inplace=True)
df_mensual.head()

,COMP_S_NAME,PROD_DT,VO_OIL_PROD,VO_GAS_PROD,VO_WAT_PROD,DIAS_ON
0,SCH-002BTI,2007-11-30,9387.29,2816.19,1515.49,30.0
1,SCH-002BTI,2007-12-31,10136.88,3041.06,1317.22,31.0
2,SCH-002BTI,2008-01-31,8469.48,2540.84,1046.79,31.0
3,SCH-002BTI,2008-02-29,7967.25,2390.18,1148.60,29.0
4,SCH-002BTI,2008-03-31,8668.35,2600.51,1238.34,31.0


In [101]:
# Delete the rows with have 'PLAN' in the COMP_S_NAME column
df_mensual = df_mensual[~df_mensual['COMP_S_NAME'].str.contains('PL')]
df_mensual.head()

,COMP_S_NAME,PROD_DT,VO_OIL_PROD,VO_GAS_PROD,VO_WAT_PROD,DIAS_ON
0,SCH-002BTI,2007-11-30,9387.29,2816.19,1515.49,30.0
1,SCH-002BTI,2007-12-31,10136.88,3041.06,1317.22,31.0
2,SCH-002BTI,2008-01-31,8469.48,2540.84,1046.79,31.0
3,SCH-002BTI,2008-02-29,7967.25,2390.18,1148.60,29.0
4,SCH-002BTI,2008-03-31,8668.35,2600.51,1238.34,31.0


In [102]:
# rename columns
df_mensual.rename(columns={'COMP_S_NAME': 'UNIQUEID', 'PROD_DT': 'Date',
                        'DIAS_ON': 'DAYS', 'VO_OIL_PROD': 'OILP', 'VO_WAT_PROD': 'WATP', 'VO_GAS_PROD': 'GASP'}, inplace=True)

# Compute the cumulative productions OIL_cum	GAS_cum	WAT_cum
df_mensual['OIL_cum'] = df_mensual.groupby(['UNIQUEID'])['OILP'].cumsum()
df_mensual['GAS_cum'] = df_mensual.groupby(['UNIQUEID'])['GASP'].cumsum()
df_mensual['WAT_cum'] = df_mensual.groupby(['UNIQUEID'])['WATP'].cumsum()


df_mensual.head()

,UNIQUEID,Date,OILP,GASP,WATP,DAYS,OIL_cum,GAS_cum,WAT_cum
0,SCH-002BTI,2007-11-30,9387.29,2816.19,1515.49,30.0,9387.29,2816.19,1515.49
1,SCH-002BTI,2007-12-31,10136.88,3041.06,1317.22,31.0,19524.17,5857.25,2832.71
2,SCH-002BTI,2008-01-31,8469.48,2540.84,1046.79,31.0,27993.65,8398.09,3879.50
3,SCH-002BTI,2008-02-29,7967.25,2390.18,1148.60,29.0,35960.90,10788.27,5028.10
4,SCH-002BTI,2008-03-31,8668.35,2600.51,1238.34,31.0,44629.25,13388.78,6266.44


In [103]:
df_mensual['Date'].dt.year.unique()

array([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019, 2020, 2021, 2022, 2023, 2003, 2004, 2005, 2006, 2001,
       2002, 1978, 1979, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 1972, 1973, 1977, 1980,
       1981, 1982, 1983, 1984, 1985, 1974, 1976, 1975], dtype=int64)

### MERGE DATA FRAMES
In order to filter the data of ```Lower T Sandstone``` and ```Lower U Sandstone```

In [104]:
merged_df = pd.merge(df_mensual, df_maestra[['COMPLETION_LEGAL_NAME', 'WELLBORE_ID', 'RSVR_NAME']],
                    left_on='UNIQUEID', right_on='COMPLETION_LEGAL_NAME', how='left')

merged_df.head()

,UNIQUEID,Date,OILP,GASP,WATP,DAYS,OIL_cum,GAS_cum,WAT_cum,COMPLETION_LEGAL_NAME,WELLBORE_ID,RSVR_NAME
0,SCH-002BTI,2007-11-30,9387.29,2816.19,1515.49,30.0,9387.29,2816.19,1515.49,SCH-002BTI,S-2B,LOWER T
1,SCH-002BTI,2007-12-31,10136.88,3041.06,1317.22,31.0,19524.17,5857.25,2832.71,SCH-002BTI,S-2B,LOWER T
2,SCH-002BTI,2008-01-31,8469.48,2540.84,1046.79,31.0,27993.65,8398.09,3879.50,SCH-002BTI,S-2B,LOWER T
3,SCH-002BTI,2008-02-29,7967.25,2390.18,1148.60,29.0,35960.90,10788.27,5028.10,SCH-002BTI,S-2B,LOWER T
4,SCH-002BTI,2008-03-31,8668.35,2600.51,1238.34,31.0,44629.25,13388.78,6266.44,SCH-002BTI,S-2B,LOWER T


In [105]:
# verify the reservoir names
merged_df['RSVR_NAME'].unique()

array(['LOWER T', 'LOWER U', 'UPPER U', 'TIYUYACU', nan, 'UPPER T'],
      dtype=object)

In [106]:
# Verify the number of wells of TIYUYACU reservoir
merged_df[merged_df['RSVR_NAME'] == 'TIYUYACU']['UNIQUEID'].unique()

array(['SCH-023ITY', 'SCH-029ITY', 'SCH-084ITY', 'SCH-086ITY',
       'SCH-117ITY'], dtype=object)

In [107]:
# Drop the rows with has TIYUYACU reservoir
df_mensual_Ui_Ti = merged_df[merged_df['RSVR_NAME'] != 'TIYUYACU']

# Drop the columns we don't need
df_mensual_Ui_Ti.drop(['COMPLETION_LEGAL_NAME', 'WELLBORE_ID', 'RSVR_NAME'], axis=1, inplace=True)

df_mensual_Ui_Ti.head()

C:\Users\brand\AppData\Local\Temp\ipykernel_19912\2802887905.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mensual_Ui_Ti.drop(['COMPLETION_LEGAL_NAME', 'WELLBORE_ID', 'RSVR_NAME'], axis=1, inplace=True)


,UNIQUEID,Date,OILP,GASP,WATP,DAYS,OIL_cum,GAS_cum,WAT_cum
0,SCH-002BTI,2007-11-30,9387.29,2816.19,1515.49,30.0,9387.29,2816.19,1515.49
1,SCH-002BTI,2007-12-31,10136.88,3041.06,1317.22,31.0,19524.17,5857.25,2832.71
2,SCH-002BTI,2008-01-31,8469.48,2540.84,1046.79,31.0,27993.65,8398.09,3879.50
3,SCH-002BTI,2008-02-29,7967.25,2390.18,1148.60,29.0,35960.90,10788.27,5028.10
4,SCH-002BTI,2008-03-31,8668.35,2600.51,1238.34,31.0,44629.25,13388.78,6266.44


In [108]:
names_inject = list(df_iny_diaria['COMPLETION_NAME'].unique())
names_inject = ['SCH-090IU' if name == 'SCH-090U' else name for name in names_inject]

df_mensual_Ui_Ti = df_mensual_Ui_Ti[~df_mensual_Ui_Ti['UNIQUEID'].isin(names_inject)]

## Diaria

In [109]:
# Order the DataFrame by the well name and the date
df_diaria.sort_values(by=['COMPLETION_NAME', 'PROD_DATE'], inplace=True)
df_diaria.reset_index(drop=True, inplace=True)

# Delete the rows with have 'PLAN' in the COMPLETION_NAME column
df_diaria = df_diaria[~df_diaria['COMPLETION_NAME'].str.contains('PL')]
df_diaria.head()

,COMPLETION_NAME,PROD_DATE,HORES_ON,RATE_OIL,RATE_GAS,RATE_WAT
0,SCH-002BTI,2023-03-21,0.0,0.0,0.0,0.0
1,SCH-002BTI,2023-03-22,0.0,0.0,0.0,0.0
2,SCH-002BTI,2023-03-23,0.0,0.0,0.0,0.0
3,SCH-002BTI,2023-03-24,0.0,0.0,0.0,0.0
4,SCH-002BTI,2023-03-25,0.0,0.0,0.0,0.0


In [110]:
# rename columns
df_diaria.rename(columns={'COMPLETION_NAME': 'UNIQUEID', 'PROD_DATE': 'Date',
                        'RATE_OIL': 'OILP', 'RATE_WAT': 'WATP', 'RATE_GAS': 'GASP'}, inplace=True)
df_diaria.head()

,UNIQUEID,Date,HORES_ON,OILP,GASP,WATP
0,SCH-002BTI,2023-03-21,0.0,0.0,0.0,0.0
1,SCH-002BTI,2023-03-22,0.0,0.0,0.0,0.0
2,SCH-002BTI,2023-03-23,0.0,0.0,0.0,0.0
3,SCH-002BTI,2023-03-24,0.0,0.0,0.0,0.0
4,SCH-002BTI,2023-03-25,0.0,0.0,0.0,0.0


In [111]:
df_diaria['Date'].dt.year.unique()

array([2023, 2016, 2017, 2018, 2019, 2020, 2021, 2022], dtype=int64)

# INYECCION

# Diaria

In [112]:
# Order the DataFrame by the well name and the date
df_iny_diaria.sort_values(by=['COMPLETION_NAME', 'PROD_DATE'], inplace=True)
df_iny_diaria.reset_index(drop=True, inplace=True)

df_iny_diaria.head()

,COMPLETION_NAME,PROD_DATE,HORES_ON,RATE_WAT_INJ,WELL_INJ_PRESSURE,PUMP_PRESS,TEMPERATURE,OIL_IN_WATER,SUSPENDED_SOLIDS,OXYGEN,...,SKIMER_TURBIDITY,SKIMER_OXYGEN,BOOSTER_SUSPENDED_SOLIDS,BOOSTER_OIL_IN_WATER,BOOSTER_TURBIDITY,BOOSTER_OXYGEN,BOOSTER_SALINITY,BOOSTER_SLOPE,BOOSTER_PLUGGING_INDEX,COMMENTS
0,SCH-005IT,2009-03-01,24.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,SCH-005IT,2009-03-02,24.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,SCH-005IT,2009-03-03,24.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,SCH-005IT,2009-03-04,24.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,SCH-005IT,2009-03-05,24.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [113]:
df_iny_diaria['PROD_DATE'].dt.year.unique()

array([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021, 2022, 2023], dtype=int64)

In [114]:
df_iny_diaria.columns

Index(['COMPLETION_NAME', 'PROD_DATE', 'HORES_ON', 'RATE_WAT_INJ',
       'WELL_INJ_PRESSURE', 'PUMP_PRESS', 'TEMPERATURE', 'OIL_IN_WATER',
       'SUSPENDED_SOLIDS', 'OXYGEN', 'SALINITY', 'PLUGGING_INDEX', 'TURBIDITY',
       'SLOPE', 'WTK_SUSPENDED_SOLIDS', 'WTK_OIL_IN_WATER', 'WTK_TURBIDITY',
       'WTK_OXYGEN', 'SKIMER_SUSPENDED_SOLIDS', 'SKIMER_OIL_IN_WATER',
       'SKIMER_TURBIDITY', 'SKIMER_OXYGEN', 'BOOSTER_SUSPENDED_SOLIDS',
       'BOOSTER_OIL_IN_WATER', 'BOOSTER_TURBIDITY', 'BOOSTER_OXYGEN',
       'BOOSTER_SALINITY', 'BOOSTER_SLOPE', 'BOOSTER_PLUGGING_INDEX',
       'COMMENTS'],
      dtype='object')

In [115]:
# this information belong to updated report to 11/2023
# the well 'SCHAG-396IUI' isn't in the report
injector_names = [
    "SCH-048", "SCH-015", "SCH-076", "SCH-105I", "SCH-005", "SCH-090", "SCH-036", "SCH-103",
    "SCHI-200", "SCHI-067B", "SCHAD-361", "SCHAD-363S1", "SCH-045B", "SCHAF-382", "SCHAE-377",
    "SCHAB-315", "SCHS-261", "SCHAK-431", "SCHAA-307", "SCHM-340", "SCHE-232", "SCHAG-396IUI"]

In [116]:
# Wells which are in the df_iny_diaria['COMPLETION_NAME'] column and injector_names

# Empty dictionary to store matching injectors
matching_injectors = {}

# Iterate through each 'COMPLETION_NAME' in df_iny_diaria
for uniqueid in df_iny_diaria['COMPLETION_NAME']:
    # Check if the current 'COMPLETION_NAME' starts with any injector_name
    for injector_name in injector_names:
        if uniqueid.startswith(injector_name):
            # If the injector_name is not in the dictionary, add it with an empty set
            if injector_name not in matching_injectors:
                matching_injectors[injector_name] = set()
            # Add the current 'COMPLETION_NAME' to the set of matching injectors for the current injector_name
            matching_injectors[injector_name].add(uniqueid)

# Convert sets to lists before printing the dictionary
matching_injectors = {key: list(value) for key, value in matching_injectors.items()}

print("Dictionary with matching injectors:")
for key, values in matching_injectors.items():
    print(f"{key}: {values}")


Dictionary with matching injectors:
SCH-005: ['SCH-005IU', 'SCH-005IT']
SCH-015: ['SCH-015IU']
SCH-036: ['SCH-036IUI']
SCH-045B: ['SCH-045BIUI']
SCH-048: ['SCH-048IU', 'SCH-048IT']
SCH-076: ['SCH-076IT', 'SCH-076IU']
SCH-090: ['SCH-090IU']
SCH-103: ['SCH-103IUI']
SCH-105I: ['SCH-105IT', 'SCH-105IU']
SCHAB-315: ['SCHAB-315IUI']
SCHAD-361: ['SCHAD-361IUI']
SCHAD-363S1: ['SCHAD-363S1IUI']
SCHAE-377: ['SCHAE-377IUI']
SCHAF-382: ['SCHAF-382IUI']
SCHAG-396IUI: ['SCHAG-396IUI']
SCHAK-431: ['SCHAK-431IUI']
SCHE-232: ['SCHE-232ITI']
SCHI-067B: ['SCHI-067BIUI']
SCHI-200: ['SCHI-200IUI']
SCHM-340: ['SCHM-340ITI']
SCHS-261: ['SCHS-261IUI']


In [117]:
# Wells which are in the df_iny_diaria['COMPLETION_NAME'] column and not in injector_names

# Create a list of 'COMPLETION_NAME' values that did not match any injector_name
not_matching_injectors = [uniqueid for uniqueid in df_iny_diaria['COMPLETION_NAME'] if not any(
                        injector_name in uniqueid for injector_name in matching_injectors)]

# Remove duplicates by converting the list to a set and then back to a list
not_matching_injectors = list(set(not_matching_injectors))

print('Disposal Wells')
not_matching_injectors

Disposal Wells


['SCH-086ITY',
 'SCH-023ITY',
 'SCH-117ITY',
 'SCHI-201IH',
 'SCH-084ITY',
 'SCHB-221HTY',
 'SCH-029ITY']

In [118]:
'''
We may delete the rows with the wells which are not in the injector_names list, but we will lose
the information of the disposal wells
'''

# Delete the rows with the wells which are not in the injector_names list
# df_iny_diaria = df_iny_diaria[~df_iny_diaria['COMPLETION_NAME'].isin(not_matching_injectors)]

# df_iny_diaria.head()

'\nWe may delete the rows with the wells which are not in the injector_names list, but we will lose\nthe information of the disposal wells\n'

In [119]:
# rename columns
df_iny_diaria = df_iny_diaria[['COMPLETION_NAME', 'PROD_DATE', 'HORES_ON', 'RATE_WAT_INJ', 'WELL_INJ_PRESSURE']].rename(
                                columns={'COMPLETION_NAME': 'Pozo', 'PROD_DATE': 'Date', 
                                        'RATE_WAT_INJ': 'Winj', 'WELL_INJ_PRESSURE': 'Pia'})
df_iny_diaria.head()

,Pozo,Date,HORES_ON,Winj,Pia
0,SCH-005IT,2009-03-01,24.0,0.0,NaN
1,SCH-005IT,2009-03-02,24.0,0.0,NaN
2,SCH-005IT,2009-03-03,24.0,0.0,NaN
3,SCH-005IT,2009-03-04,24.0,0.0,NaN
4,SCH-005IT,2009-03-05,24.0,0.0,NaN


In [120]:
merged_df_iny = pd.merge(df_iny_diaria, df_sc[['COMPLETION_LEGAL_NAME', 'RESERVORIO']],
                    left_on='Pozo', right_on='COMPLETION_LEGAL_NAME', how = 'left')

# make a copy of the merged_df_iny DataFrame
df_iny_diaria = merged_df_iny.copy()

# delete 'COMPLETION_LEGAL_NAME', 'WELL_LEGAL_NAME' and 'HORES_ON' columns
df_iny_diaria.drop(['COMPLETION_LEGAL_NAME', 'HORES_ON'], axis=1, inplace=True)

# rename columns
df_iny_diaria.rename(columns={'RESERVORIO': 'Capa'}, inplace=True)

# change values in capa column, where T to TI, U to UI, U-I to UI
df_iny_diaria['Capa'] = df_iny_diaria['Capa'].replace({'T': 'TI', 'U': 'UI', 'U-I': 'UI'})

# Calculate the accumulated water injection WATER_INJ_.CUM
df_iny_diaria['WATER_INJ_.CUM'] = df_iny_diaria.groupby(['Pozo'])['Winj'].cumsum()

df_iny_diaria.head()

,Pozo,Date,Winj,Pia,Capa,WATER_INJ_.CUM
0,SCH-005IT,2009-03-01,0.0,NaN,TI,0.0
1,SCH-005IT,2009-03-02,0.0,NaN,TI,0.0
2,SCH-005IT,2009-03-03,0.0,NaN,TI,0.0
3,SCH-005IT,2009-03-04,0.0,NaN,TI,0.0
4,SCH-005IT,2009-03-05,0.0,NaN,TI,0.0


In [121]:
# names of the layers for injection and re-injection process
df_iny_diaria['Capa'].unique()

array(['TI', 'UI', 'TY', nan], dtype=object)

In [122]:
# Visualize the wells which don't have a layer
df_iny_diaria[df_iny_diaria['Capa'].isna()]['Pozo'].unique()

array(['SCHB-221HTY', 'SCHI-201IH'], dtype=object)

In [123]:
# Assign the layer to the wells cause the layer is in df_SC
df_iny_diaria.loc[df_iny_diaria['Pozo'] == 'SCHAD-361IUI', 'Capa'] = 'UI'
df_iny_diaria.loc[df_iny_diaria['Pozo'] == 'SCHI-067BIUI', 'Capa'] = 'UI'
df_iny_diaria.loc[df_iny_diaria['Pozo'] == 'SCHI-200IUI', 'Capa'] = 'UI'

df_iny_diaria.loc[df_iny_diaria['Pozo'] == 'SCHE-232ITI', 'Capa'] = 'TI'

df_iny_diaria.loc[df_iny_diaria['Pozo'] == 'SCHB-221HTY', 'Capa'] = 'TY'

df_iny_diaria.loc[df_iny_diaria['Pozo'] == 'SCHI-201IH', 'Capa'] = 'H'

# change name of the wells to determine as injector
df_iny_diaria.loc[df_iny_diaria['Pozo'] == 'SCH-090U', 'Pozo'] = 'SCH-090IU'

df_iny_diaria.head()

,Pozo,Date,Winj,Pia,Capa,WATER_INJ_.CUM
0,SCH-005IT,2009-03-01,0.0,NaN,TI,0.0
1,SCH-005IT,2009-03-02,0.0,NaN,TI,0.0
2,SCH-005IT,2009-03-03,0.0,NaN,TI,0.0
3,SCH-005IT,2009-03-04,0.0,NaN,TI,0.0
4,SCH-005IT,2009-03-05,0.0,NaN,TI,0.0


In [124]:
df_iny_diaria['Capa'].unique()

array(['TI', 'UI', 'TY', 'H'], dtype=object)

## MENSUAL

In [125]:
merged_df_iny.head()

,Pozo,Date,HORES_ON,Winj,Pia,COMPLETION_LEGAL_NAME,RESERVORIO
0,SCH-005IT,2009-03-01,24.0,0.0,NaN,SCH-005IT,TI
1,SCH-005IT,2009-03-02,24.0,0.0,NaN,SCH-005IT,TI
2,SCH-005IT,2009-03-03,24.0,0.0,NaN,SCH-005IT,TI
3,SCH-005IT,2009-03-04,24.0,0.0,NaN,SCH-005IT,TI
4,SCH-005IT,2009-03-05,24.0,0.0,NaN,SCH-005IT,TI


In [126]:
# 
df_iny_mensual = merged_df_iny[['Pozo', 'RESERVORIO', 'Date', 'HORES_ON', 'Winj', 'Pia']]
df_iny_mensual['DAYS'] = df_iny_mensual['HORES_ON']/24        # convert hours to days

df_iny_mensual.head()

,Pozo,RESERVORIO,Date,HORES_ON,Winj,Pia,DAYS
0,SCH-005IT,TI,2009-03-01,24.0,0.0,NaN,1.0
1,SCH-005IT,TI,2009-03-02,24.0,0.0,NaN,1.0
2,SCH-005IT,TI,2009-03-03,24.0,0.0,NaN,1.0
3,SCH-005IT,TI,2009-03-04,24.0,0.0,NaN,1.0
4,SCH-005IT,TI,2009-03-05,24.0,0.0,NaN,1.0


In [127]:
df_iny_mensual['Date'].dt.year.unique()

array([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021, 2022, 2023], dtype=int64)

In [128]:
# Filter pressure values greater than 0 and not null
df_iny_mensual['Pia_Days'] = df_iny_mensual['Pia'].apply(lambda x: 1 if x > 0 and not pd.isnull(x) else 0)

# Calculate the accumulated values by month
df_iny_mensual['Month'] = df_iny_mensual.groupby(['Pozo', df_iny_mensual['Date'].dt.to_period('M')])['Date'].transform('min')
df_iny_month = df_iny_mensual.groupby(['Pozo', 'Month'])[['Winj', 'DAYS', 'Pia', 'Pia_Days', 'RESERVORIO']].sum().reset_index()

# Pressure average by month
df_iny_month['Pia_Avg'] = df_iny_month['Pia'] / df_iny_month['Pia_Days']

df_iny_month.head()

C:\Users\brand\AppData\Local\Temp\ipykernel_19912\3070020238.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_iny_month = df_iny_mensual.groupby(['Pozo', 'Month'])[['Winj', 'DAYS', 'Pia', 'Pia_Days', 'RESERVORIO']].sum().reset_index()


,Pozo,Month,Winj,DAYS,Pia,Pia_Days,Pia_Avg
0,SCH-005IT,2009-03-01,0.0,31.0,0.0,0,NaN
1,SCH-005IT,2009-07-01,0.0,31.0,0.0,0,NaN
2,SCH-005IT,2009-09-01,0.0,30.0,0.0,0,NaN
3,SCH-005IT,2009-11-01,128971.0,30.0,0.0,0,NaN
4,SCH-005IT,2009-12-01,134078.0,31.0,0.0,0,NaN


In [129]:
# Filter pressure values greater than 0 and not null
df_iny_mensual['Pia_Days'] = df_iny_mensual['Pia'].apply(lambda x: 1 if x > 0 and not pd.isnull(x) else 0)

# Calculate the accumulated values by month
df_iny_mensual['Month'] = df_iny_mensual.groupby(['Pozo', df_iny_mensual['Date'].dt.to_period('M')])['Date'].transform('min')
df_iny_month = df_iny_mensual.groupby(['Pozo', 'Month']).agg({'Winj': 'sum', 'DAYS': 'sum', 'Pia': 'sum', 'Pia_Days': 'sum', 'RESERVORIO': 'first'}).reset_index()

# Pressure average by month
df_iny_month['Pia_Avg'] = df_iny_month['Pia'] / df_iny_month['Pia_Days']

df_iny_month.head()

,Pozo,Month,Winj,DAYS,Pia,Pia_Days,RESERVORIO,Pia_Avg
0,SCH-005IT,2009-03-01,0.0,31.0,0.0,0,TI,NaN
1,SCH-005IT,2009-07-01,0.0,31.0,0.0,0,TI,NaN
2,SCH-005IT,2009-09-01,0.0,30.0,0.0,0,TI,NaN
3,SCH-005IT,2009-11-01,128971.0,30.0,0.0,0,TI,NaN
4,SCH-005IT,2009-12-01,134078.0,31.0,0.0,0,TI,NaN


In [130]:
# rename column
df_iny_month.rename(columns={'Pozo': 'UNIQUEID', 'RESERVORIO': 'CAPA', 'Month': 'Date', 'Winj': 'WATER_INJ_VOL', 'Pia_Avg': 'WHP'}, inplace=True)

#create a new column INJ_TYPE
df_iny_month['INJ_TYPE'] = 'water'

# Calculate the accumulated water injection WATER_INJ_.CUM
df_iny_month['WATER_INJ_.CUM'] = df_iny_month.groupby(['UNIQUEID'])['WATER_INJ_VOL'].cumsum()

df_iny_month = df_iny_month[['UNIQUEID', 'CAPA', 'Date', 'INJ_TYPE', 'WATER_INJ_VOL', 'DAYS', 'WHP', 'WATER_INJ_.CUM']]

df_iny_month.head()

,UNIQUEID,CAPA,Date,INJ_TYPE,WATER_INJ_VOL,DAYS,WHP,WATER_INJ_.CUM
0,SCH-005IT,TI,2009-03-01,water,0.0,31.0,NaN,0.0
1,SCH-005IT,TI,2009-07-01,water,0.0,31.0,NaN,0.0
2,SCH-005IT,TI,2009-09-01,water,0.0,30.0,NaN,0.0
3,SCH-005IT,TI,2009-11-01,water,128971.0,30.0,NaN,128971.0
4,SCH-005IT,TI,2009-12-01,water,134078.0,31.0,NaN,263049.0


In [131]:
df_iny_month['CAPA'].unique()

array(['TI', 'UI', 'TY', None], dtype=object)

In [132]:
# dame aquellos que tienen None
df_iny_month[df_iny_month['CAPA'].isna()]['UNIQUEID'].unique()

array(['SCHB-221HTY', 'SCHI-201IH'], dtype=object)

In [133]:
# Assign the layer to the wells cause the layer is in df_SC
df_iny_month.loc[df_iny_month['UNIQUEID'] == 'SCHAD-361IUI', 'CAPA'] = 'UI'
df_iny_month.loc[df_iny_month['UNIQUEID'] == 'SCHI-067BIUI', 'CAPA'] = 'UI'
df_iny_month.loc[df_iny_month['UNIQUEID'] == 'SCHI-200IUI', 'CAPA'] = 'UI'

df_iny_month.loc[df_iny_month['UNIQUEID'] == 'SCHE-232ITI', 'CAPA'] = 'TI'

df_iny_month.loc[df_iny_month['UNIQUEID'] == 'SCHB-221HTY', 'CAPA'] = 'TY'

df_iny_month.loc[df_iny_month['UNIQUEID'] == 'SCHI-201IH', 'CAPA'] = 'H'

# change name of the wells to determine as injector
df_iny_month.loc[df_iny_month['UNIQUEID'] == 'SCH-090U', 'UNIQUEID'] = 'SCH-090IU'

df_iny_month.head()

,UNIQUEID,CAPA,Date,INJ_TYPE,WATER_INJ_VOL,DAYS,WHP,WATER_INJ_.CUM
0,SCH-005IT,TI,2009-03-01,water,0.0,31.0,NaN,0.0
1,SCH-005IT,TI,2009-07-01,water,0.0,31.0,NaN,0.0
2,SCH-005IT,TI,2009-09-01,water,0.0,30.0,NaN,0.0
3,SCH-005IT,TI,2009-11-01,water,128971.0,30.0,NaN,128971.0
4,SCH-005IT,TI,2009-12-01,water,134078.0,31.0,NaN,263049.0


### Filtered data to load in OFM

In [134]:
# Suponiendo que 'df' es tu DataFrame
df_iny_month_OFM = df_iny_month[~((df_iny_month['WATER_INJ_VOL'] == 0) & df_iny_month['WHP'].isnull())]
df_iny_month_OFM.drop(['INJ_TYPE'], axis=1, inplace=True)
df_iny_month_OFM.rename(columns={'UNIQUEID': 'Pozo', 'Date': 'Month', 'WATER_INJ_VOL': 'Winj', 'WHP': 'Pia_Avg'}, inplace=True)
df_iny_month_OFM.head()

C:\Users\brand\AppData\Local\Temp\ipykernel_19912\1942761519.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_iny_month_OFM.drop(['INJ_TYPE'], axis=1, inplace=True)
C:\Users\brand\AppData\Local\Temp\ipykernel_19912\1942761519.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_iny_month_OFM.rename(columns={'UNIQUEID': 'Pozo', 'Date': 'Month', 'WATER_INJ_VOL': 'Winj', 'WHP': 'Pia_Avg'}, inplace=True)


,Pozo,CAPA,Month,Winj,DAYS,Pia_Avg,WATER_INJ_.CUM
3,SCH-005IT,TI,2009-11-01,128971.0,30.0,NaN,128971.0
4,SCH-005IT,TI,2009-12-01,134078.0,31.0,NaN,263049.0
5,SCH-005IT,TI,2010-01-01,137696.0,31.0,NaN,400745.0
6,SCH-005IT,TI,2010-02-01,139108.0,28.0,NaN,539853.0
7,SCH-005IT,TI,2010-03-01,137382.0,31.0,NaN,677235.0


# EVENTOS - PERFORADOS

## Notas

In [135]:
# Import the Excel file
excel_file_eventos = r'C:\Users\brand\OneDrive - Universidad Central del Ecuador\Documentos\Maestría EOR\Tesis EOR\Info EPPEC\2018_INFORMACION POZOS SACHA-EVENTOS -INTERVALOS_PAM.xlsx'

# Read the Excel file into a DataFrame
df_eventos = pd.read_excel(excel_file_eventos, sheet_name='POZOS EVENTOS', skiprows=6)

df_eventos.head()

,BLOQUE,ACTIVO,POZO,EVENTO,COD EVENTO,TRABAJO,FECHA INICIO,FECHA FIN
0,B60,SA,SCHA-001,DEV DRILLING,DRL,DRL,1969-01-21,1969-02-18 00:00:00
1,B60,SA,SCHA-001,DEV COMPLETION,COM,COM,1969-02-22,1969-02-25 00:00:00
2,B60,SA,SCH-002,DEV DRILLING,DRL,DRL,1969-07-21,1969-08-22 00:00:00
3,B60,SA,SCH-002,DEV COMPLETION,COM,COM,1969-08-29,1969-08-31 00:00:00
4,B60,SA,SCH-003,DEV DRILLING,DRL,DRL,1969-09-04,1969-10-03 00:00:00


In [136]:
# Generate a merge to change the name of the wells
df_eventos = pd.merge(df_eventos[['POZO', 'FECHA FIN', 'TRABAJO', 'EVENTO']], df_wells_merge[['WELL_LEGAL_NAME', 'NOMBRE_COMPLETO']],
                    left_on='POZO', right_on='WELL_LEGAL_NAME', how = 'left')

# Water production wells from report to 11/2023
well_water_prod = ['SCHAA-306', 'SCHAD-366', 'SCHAE-374', 'SCHAF-380', 'SCHAB-318', 'SCHAK-432', 'SCHE-166']

# if the well is nan in NOMBRE_COMPLETO column, then put the name of the well from POZO, just in the wells which are in well_water_prod list
df_eventos['NOMBRE_COMPLETO'] = np.where(df_eventos['NOMBRE_COMPLETO'].isnull() & df_eventos['POZO'].isin(
                        well_water_prod), df_eventos['POZO'], df_eventos['NOMBRE_COMPLETO'])

# change the name of the wells to desired name format
df_eventos['NOMBRE_COMPLETO'] = df_eventos['NOMBRE_COMPLETO'].replace(
                                {'SCHAA-306': 'S-AA306', 'SCHAD-366': 'S-AD366', 'SCHAE-374': 'S-AE374', 'SCHAF-380': 'S-AF380',
                                'SCHAB-318': 'S-AB318', 'SCHAK-432': 'S-AK432', 'SCHE-166': 'S-E166'})

# if nan in NOMBRE_COMPLETO column, delete the row
df_eventos = df_eventos[~df_eventos['NOMBRE_COMPLETO'].isnull()]

# Put in the desired format
df_eventos = df_eventos[['NOMBRE_COMPLETO', 'FECHA FIN', 'TRABAJO', 'EVENTO']]
df_eventos.rename(columns={'NOMBRE_COMPLETO': 'IDENTIFICADOR', 'FECHA FIN': 'FECHA',
                        'TRABAJO': 'CATEGORÍA', 'EVENTO': 'DATO'}, inplace=True)
df_eventos.head()

,IDENTIFICADOR,FECHA,CATEGORÍA,DATO
0,S-A01,1969-02-18 00:00:00,DRL,DEV DRILLING
1,S-A01,1969-02-25 00:00:00,COM,DEV COMPLETION
2,S-2,1969-08-22 00:00:00,DRL,DEV DRILLING
3,S-2,1969-08-31 00:00:00,COM,DEV COMPLETION
4,S-3,1969-10-03 00:00:00,DRL,DEV DRILLING


## Perforados

In [137]:
# Import the Excel file
excel_file_perf = r'C:\Users\brand\OneDrive - Universidad Central del Ecuador\Documentos\Maestría EOR\Tesis EOR\Info EPPEC\NUEVA INFORMACIÓN 06112023\PRF-SA-RPR-231108-INTERVALOS SACHA.xlsx'

# Read the Excel file into a DataFrame
df_perf = pd.read_excel(excel_file_perf, sheet_name='INTERVALOS CAÑONEADOS', skiprows=13)

df_perf.head()

,BLOQUE,CAMPO,ACTIVO,NOMBRE LARGO,POZO,WELLBORE,FECHA CAÑONEO,Top MD (ft),Btm MD (ft),RAZON,ESTATUS,ARENA
0,B60,SACHA,SA,SACHA-003,SCH-003,SCH-003,1969-10-08,9730.0,9737.0,PRODUCTION,OPEN,"ARENA ""T"""
1,B60,SACHA,SA,SACHA-003,SCH-003,SCH-003,1969-10-08,9954.0,9982.0,PRODUCTION,OPEN,HOLLIN INFERIOR
2,B60,SACHA,SA,SACHA-003,SCH-003,SCH-003,1969-10-08,9740.0,9746.0,PRODUCTION,OPEN,"ARENA ""T"""
3,B60,SACHA,SA,SACHA-003,SCH-003,SCH-003,1969-10-08,9935.0,9939.0,PRODUCTION,OPEN,HOLLIN INFERIOR
4,B60,SACHA,SA,SACHA-003,SCH-003,SCH-003,1969-10-08,9940.0,9952.0,PRODUCTION,OPEN,HOLLIN INFERIOR


In [138]:
# Generate a merge to change the name of the wells
df_perf = pd.merge(df_perf[['POZO', 'FECHA CAÑONEO', 'Top MD (ft)', 'Btm MD (ft)', 'RAZON', 'ESTATUS', 'ARENA']], df_wells_merge[['WELL_LEGAL_NAME', 'NOMBRE_COMPLETO']],
                    left_on='POZO', right_on='WELL_LEGAL_NAME', how = 'left')

# if the well is nan in NOMBRE_COMPLETO column, then put the name of the well from POZO, just in the wells which are in well_water_prod list
df_perf['NOMBRE_COMPLETO'] = np.where(df_perf['NOMBRE_COMPLETO'].isnull() & df_perf['POZO'].isin(
                        well_water_prod), df_perf['POZO'], df_perf['NOMBRE_COMPLETO'])

# change the name of the wells to desired name format
df_perf['NOMBRE_COMPLETO'] = df_perf['NOMBRE_COMPLETO'].replace(
                                {'SCHAA-306': 'S-AA306', 'SCHAD-366': 'S-AD366', 'SCHAE-374': 'S-AE374', 'SCHAF-380': 'S-AF380',
                                'SCHAB-318': 'S-AB318', 'SCHAK-432': 'S-AK432', 'SCHE-166': 'S-E166'})

# Change the punzados types
df_perf['RAZON'] = df_perf['RAZON'].replace(
                                {'CEMENT SQUEEZE': 'SQZ', 'PRODUCTION  "TI"': 'PRODUCTION', 'ARENA INYECTORA': 'INJECTION', 'PRODUCTION  "UIf"': 'PRODUCTION', 'PRODUCTION  "UI"': 'PRODUCTION'})

# remove duplicated rows
df_perf.drop_duplicates(inplace=True)

# generate a copy of the df_perf DataFrame
df_perf_original = df_perf.copy()

# delete the rows with nan
df_perf = df_perf[~df_perf['NOMBRE_COMPLETO'].isnull()]

# Put in the desired format
# TODO: check if the columns are correct and their names
df_perf = df_perf[['NOMBRE_COMPLETO', 'FECHA CAÑONEO', 'Top MD (ft)', 'Btm MD (ft)', 'RAZON']].rename(
                columns={'NOMBRE_COMPLETO': 'IDENTIFICADOR', 'FECHA CAÑONEO': 'FECHA',
                        'Top MD (ft)': 'TOPE', 'Btm MD (ft)': 'BASE', 'RAZON': 'TIPO_DE_PUNZADO'})
df_perf.head()

,IDENTIFICADOR,FECHA,TOPE,BASE,TIPO_DE_PUNZADO
0,S-3,1969-10-08,9730.0,9737.0,PRODUCTION
1,S-3,1969-10-08,9954.0,9982.0,PRODUCTION
2,S-3,1969-10-08,9740.0,9746.0,PRODUCTION
3,S-3,1969-10-08,9935.0,9939.0,PRODUCTION
4,S-3,1969-10-08,9940.0,9952.0,PRODUCTION


## Notas 2

In [139]:
df_mensual_Ui_Ti.head()

,UNIQUEID,Date,OILP,GASP,WATP,DAYS,OIL_cum,GAS_cum,WAT_cum
0,SCH-002BTI,2007-11-30,9387.29,2816.19,1515.49,30.0,9387.29,2816.19,1515.49
1,SCH-002BTI,2007-12-31,10136.88,3041.06,1317.22,31.0,19524.17,5857.25,2832.71
2,SCH-002BTI,2008-01-31,8469.48,2540.84,1046.79,31.0,27993.65,8398.09,3879.50
3,SCH-002BTI,2008-02-29,7967.25,2390.18,1148.60,29.0,35960.90,10788.27,5028.10
4,SCH-002BTI,2008-03-31,8668.35,2600.51,1238.34,31.0,44629.25,13388.78,6266.44


In [140]:
# Find the last month of production for each well
last_month = df_mensual_Ui_Ti.groupby('UNIQUEID')['Date'].idxmax()

# Filter rows corresponding to the last month of each well
df_notas_prod = df_mensual_Ui_Ti.loc[last_month]

# Calculate the average of OILP for the last month divided by the number of days of the month
df_notas_prod['Qo [bopd]'] = df_notas_prod['OILP'] / df_notas_prod['DAYS']
df_notas_prod['Qt [bfpd]'] = (df_notas_prod['WATP'] + df_notas_prod['OILP']) / df_notas_prod['DAYS']

# Generate a merge to have the reservoir name
df_notas_prod = pd.merge(df_notas_prod[['UNIQUEID', 'Date', 'Qo [bopd]', 'Qt [bfpd]', 'OIL_cum']], df_sc[['COMPLETION_LEGAL_NAME', 'RESERVORIO']],
                    left_on='UNIQUEID', right_on='COMPLETION_LEGAL_NAME', how = 'left')
# Put in the desired format
df_notas_prod.rename(columns={'UNIQUEID': 'Completion', 'Date': 'FECHA', 'OIL_cum': 'Np [Bls]'}, inplace=True)

df_notas_prod.head()

,Completion,FECHA,Qo [bopd],Qt [bfpd],Np [Bls],COMPLETION_LEGAL_NAME,RESERVORIO
0,SCH-002BTI,2023-06-30,NaN,NaN,610218.002,SCH-002BTI,TI
1,SCH-002BUI,2023-06-30,NaN,NaN,231683.190,SCH-002BUI,UI
2,SCH-002TI,2023-06-30,NaN,NaN,0.000,SCH-002TI,TI
3,SCH-002UI,2023-06-30,NaN,NaN,237197.000,SCH-002UI,UI
4,SCH-003TI,2023-06-30,NaN,NaN,2380627.342,SCH-003TI,TI


In [141]:
# Column to determine if the well is producing
df_notas_prod['PRODUCCION'] = np.where(df_notas_prod['Qt [bfpd]'] > 0, 'SI', 'NO')
df_notas_prod.head()

,Completion,FECHA,Qo [bopd],Qt [bfpd],Np [Bls],COMPLETION_LEGAL_NAME,RESERVORIO,PRODUCCION
0,SCH-002BTI,2023-06-30,NaN,NaN,610218.002,SCH-002BTI,TI,NO
1,SCH-002BUI,2023-06-30,NaN,NaN,231683.190,SCH-002BUI,UI,NO
2,SCH-002TI,2023-06-30,NaN,NaN,0.000,SCH-002TI,TI,NO
3,SCH-002UI,2023-06-30,NaN,NaN,237197.000,SCH-002UI,UI,NO
4,SCH-003TI,2023-06-30,NaN,NaN,2380627.342,SCH-003TI,TI,NO


In [142]:
# Find the last month of production for each well
last_month = df_iny_month.groupby('UNIQUEID')['Date'].idxmax()

# Filter rows corresponding to the last month of each well
df_notas_iny = df_iny_month.loc[last_month]

# Calculate the average of OILP for the last month divided by the number of days of the month
df_notas_iny['W [bwpd]'] = df_notas_iny['WATER_INJ_VOL'] / df_notas_iny['DAYS']

# Generate a merge to have the reservoir name
df_notas_iny = pd.merge(df_notas_iny[['UNIQUEID', 'Date', 'WATER_INJ_VOL', 'W [bwpd]', 'WATER_INJ_.CUM']], df_sc[['COMPLETION_LEGAL_NAME', 'RESERVORIO']],
                    left_on='UNIQUEID', right_on='COMPLETION_LEGAL_NAME', how = 'left')

# Put in the desired format
df_notas_iny.rename(columns={'UNIQUEID': 'Completion', 'Date': 'FECHA', 'OIL_cum': 'Wp [Bls]', 'WATER_INJ_.CUM': 'Wp [Bls]'}, inplace=True)

df_notas_iny.head()

,Completion,FECHA,WATER_INJ_VOL,W [bwpd],Wp [Bls],COMPLETION_LEGAL_NAME,RESERVORIO
0,SCH-005IT,2023-09-01,0.00,NaN,8.639658e+06,SCH-005IT,TI
1,SCH-005IU,2023-09-01,0.00,NaN,7.594596e+06,SCH-005IU,UI
2,SCH-015IU,2023-09-01,1451.00,1451.00,7.933394e+06,SCH-015IU,UI
3,SCH-023ITY,2023-09-01,12918.00,12918.00,4.749058e+07,SCH-023ITY,TY
4,SCH-029ITY,2023-09-01,7198.44,7198.44,4.205948e+07,SCH-029ITY,TY


In [143]:
# Column to determine if the completion is producing
df_notas_iny['PRODUCCION'] = np.where(df_notas_iny['W [bwpd]'] > 0, 'SI', 'NO')
df_notas_iny.head()

,Completion,FECHA,WATER_INJ_VOL,W [bwpd],Wp [Bls],COMPLETION_LEGAL_NAME,RESERVORIO,PRODUCCION
0,SCH-005IT,2023-09-01,0.00,NaN,8.639658e+06,SCH-005IT,TI,NO
1,SCH-005IU,2023-09-01,0.00,NaN,7.594596e+06,SCH-005IU,UI,NO
2,SCH-015IU,2023-09-01,1451.00,1451.00,7.933394e+06,SCH-015IU,UI,SI
3,SCH-023ITY,2023-09-01,12918.00,12918.00,4.749058e+07,SCH-023ITY,TY,SI
4,SCH-029ITY,2023-09-01,7198.44,7198.44,4.205948e+07,SCH-029ITY,TY,SI


In [144]:
# Append the DataFrames
df_notas_2 = df_notas_prod[['Completion', 'FECHA', 'PRODUCCION', 'RESERVORIO']].append(
                            df_notas_iny[['Completion', 'FECHA', 'PRODUCCION', 'RESERVORIO']], ignore_index=True)

df_notas_2.head()

C:\Users\brand\AppData\Local\Temp\ipykernel_19912\2900387036.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_notas_2 = df_notas_prod[['Completion', 'FECHA', 'PRODUCCION', 'RESERVORIO']].append(


,Completion,FECHA,PRODUCCION,RESERVORIO
0,SCH-002BTI,2023-06-30,NO,TI
1,SCH-002BUI,2023-06-30,NO,UI
2,SCH-002TI,2023-06-30,NO,TI
3,SCH-002UI,2023-06-30,NO,UI
4,SCH-003TI,2023-06-30,NO,TI


In [145]:
# # mege con master para obtener el nombre del pozo
# df_notas_2 = pd.merge(df_notas_2, df_wells_merge[['COMPLETION_LEGAL_NAME', 'WELL_LEGAL_NAME', 'NOMBRE_COMPLETO']],
#                     left_on='Completion', right_on='COMPLETION_LEGAL_NAME', how = 'left')
# df_notas_2.head()

In [146]:
# Make a list of the completions with reservoir nan
completions_with_reservoir_NAN = df_notas_2[df_notas_2['RESERVORIO'].isnull()]['Completion'].unique()

# Assign the reservoir to the wells cause the reservoir cause isn't in df_SC
df_notas_2.loc[df_notas_2['Completion'].isin(completions_with_reservoir_NAN) & df_notas_2['Completion'].str.contains('UI'), 'RESERVORIO'] = 'UI'
df_notas_2.loc[df_notas_2['Completion'].isin(completions_with_reservoir_NAN) & df_notas_2['Completion'].str.contains('TI'), 'RESERVORIO'] = 'TI'
df_notas_2.loc[df_notas_2['Completion'].isin(completions_with_reservoir_NAN) & df_notas_2['Completion'].str.contains('TY'), 'RESERVORIO'] = 'TY'

df_notas_2.loc[df_notas_2['Completion'].isin(['SCHD-020U', 'SCHJ-152U', 'SCHK-199U']), 'RESERVORIO'] = 'UI'
df_notas_2.loc[df_notas_2['Completion'].isin(['SCHD-020T']), 'RESERVORIO'] = 'TI'
df_notas_2.loc[df_notas_2['Completion'].isin(['SCHI-201IH']), 'RESERVORIO'] = 'H'

df_notas_2.head()

,Completion,FECHA,PRODUCCION,RESERVORIO
0,SCH-002BTI,2023-06-30,NO,TI
1,SCH-002BUI,2023-06-30,NO,UI
2,SCH-002TI,2023-06-30,NO,TI
3,SCH-002UI,2023-06-30,NO,UI
4,SCH-003TI,2023-06-30,NO,TI


In [147]:
# # Marge the DataFrames
# df_wells_merge = df_maestra.merge(df_sc, on='COMPLETION_LEGAL_NAME', how='inner')

# # change the name of the columns to NOMBRE_COMPLETO	X	Y   FCOMP   CAMPO
# df_wells_merge.rename(columns={'WELLBORE_ID':'NOMBRE_COMPLETO','COMPLETION_COORDINATE_X':'X',
#                         'COMPLETION_COORDINATE_Y':'Y', 'FECHA_COMPLETACION':'FCOMP', 'FIELD':'CAMPO'}, inplace=True)

# df_wells_merge.head()

# ESTADOS CAPA

In [148]:
df_perf_original.head()

,POZO,FECHA CAÑONEO,Top MD (ft),Btm MD (ft),RAZON,ESTATUS,ARENA,WELL_LEGAL_NAME,NOMBRE_COMPLETO
0,SCH-003,1969-10-08,9730.0,9737.0,PRODUCTION,OPEN,"ARENA ""T""",SCH-003,S-3
1,SCH-003,1969-10-08,9954.0,9982.0,PRODUCTION,OPEN,HOLLIN INFERIOR,SCH-003,S-3
2,SCH-003,1969-10-08,9740.0,9746.0,PRODUCTION,OPEN,"ARENA ""T""",SCH-003,S-3
3,SCH-003,1969-10-08,9935.0,9939.0,PRODUCTION,OPEN,HOLLIN INFERIOR,SCH-003,S-3
4,SCH-003,1969-10-08,9940.0,9952.0,PRODUCTION,OPEN,HOLLIN INFERIOR,SCH-003,S-3


In [149]:

# Generate conditions to find the names of the layers are included
condition_Ui = df_perf_original['ARENA'].str.contains('Ui|UI|ARENA U|U I|A"U"|A "U"|U INFERIOR|Lower U', case=False, na=False) & ~df_perf_original[
                                'ARENA'].str.contains('"U" SU|U SU', case=False, na=False)
condition_Ti = df_perf_original['ARENA'].str.contains('"Ti"|Ti|TI|ARENA T I|A T|T I|A"T"|A "T"|T INFERIOR|Lower T|main T', case=False, na=False) & ~df_perf_original[
                                'ARENA'].str.contains('"T" SU|T SU', case=False, na=False)
condition_H = df_perf_original['ARENA'].str.contains('HOLLIN|HOLLÍN|HI|ARENA H|A "H"', case=False, na=False)

# Replace the values in ARENA column at de desired format
df_perf_original['ARENA 2'] = np.where(condition_Ui, 'UI', df_perf_original['ARENA'])
df_perf_original['ARENA 2'] = np.where(condition_Ti, 'TI', df_perf_original['ARENA 2'])
df_perf_original['ARENA 2'] = np.where(condition_H, 'H', df_perf_original['ARENA 2'])

df_perf_original.head()

,POZO,FECHA CAÑONEO,Top MD (ft),Btm MD (ft),RAZON,ESTATUS,ARENA,WELL_LEGAL_NAME,NOMBRE_COMPLETO,ARENA 2
0,SCH-003,1969-10-08,9730.0,9737.0,PRODUCTION,OPEN,"ARENA ""T""",SCH-003,S-3,TI
1,SCH-003,1969-10-08,9954.0,9982.0,PRODUCTION,OPEN,HOLLIN INFERIOR,SCH-003,S-3,H
2,SCH-003,1969-10-08,9740.0,9746.0,PRODUCTION,OPEN,"ARENA ""T""",SCH-003,S-3,TI
3,SCH-003,1969-10-08,9935.0,9939.0,PRODUCTION,OPEN,HOLLIN INFERIOR,SCH-003,S-3,H
4,SCH-003,1969-10-08,9940.0,9952.0,PRODUCTION,OPEN,HOLLIN INFERIOR,SCH-003,S-3,H


In [150]:
# View the unique values of Arena 2
df_perf_original['ARENA 2'].unique()

array(['TI', 'H', 'UI', 'ARENA "HS"', nan, 'ARENA U SUP', 'CALIZA "C"',
       'BASAL TENA', 'ARENA "T" SUPERIOR', 'CALIZA "B"',
       'ARENA "U" SUPERIOR', 'ARENA "TS"', 'ARENA "US"', 'ARENA BT',
       'CALIZA "A"', 'U SUPERIOR', 'NAPO', 'T SUPERIOR', 'CONGLOMERADO',
       'ARENA "BT"', 'Upper T Sandstone', 'Basal Tena',
       'ARENA BASAL TENA', 'CALIZA "M1"', 'CALIZA "M2"', 'ARENA      "U"',
       'Upper U Sandstone', 'ARENA "Hs"', 'ARENA U SUPERIOR', 'Arena BT',
       'ORTEGUAZA', 'ARENA      "BT"', 'ARENA "HS" FRACTURADA',
       'TAPON BALANCEADO', 'CALIZA A ', 'ARENA NAPO T BASAL'],
      dtype=object)

In [151]:
# Put in the desired format
# TODO: Check the change of the names of the ARENA 2
df_layer_status = df_perf_original[['POZO', 'ARENA', 'ARENA 2', 'FECHA CAÑONEO', 'ESTATUS']].rename(
                columns={'POZO': 'IDENTIFICADOR', 'ARENA 2': 'CAPA', 'FECHA CAÑONEO': 'FECHA', 'ESTATUS': 'ESTADO'})
df_layer_status.head()

,IDENTIFICADOR,ARENA,CAPA,FECHA,ESTADO
0,SCH-003,"ARENA ""T""",TI,1969-10-08,OPEN
1,SCH-003,HOLLIN INFERIOR,H,1969-10-08,OPEN
2,SCH-003,"ARENA ""T""",TI,1969-10-08,OPEN
3,SCH-003,HOLLIN INFERIOR,H,1969-10-08,OPEN
4,SCH-003,HOLLIN INFERIOR,H,1969-10-08,OPEN


# INSTALACIONES

In [152]:
import pdfplumber
import re
from collections import namedtuple


In [153]:
# pdf_file = pdfplumber.open('2017_SCHP-188 WO# 08_ZS.pdf')

In [154]:
loc_colums = ['No', 'Jts', 'Tope MD', 'Tope TVD', 'Longitud', 'OD Nom', 'Descripción']

In [155]:
# with pdfplumber.open('2017_SCHP-188 WO# 08_ZS.pdf') as pdf:
#     data = pdf.pages[0].extract_tables()
#     # data = data[0][1:]
# data

# PRUEBAS DE PRODUCCIÓN

In [156]:
df_production_test.head()

,PTYPE,COMPLETION_LEGAL_NAME,TEST_DATE,T_HRS,TEST_OIL_24,TEST_GAS_24,TEST_WAT_24,GRAV_OIL API,BSW,CASING_PRESS,...,REMARKS,TEST_PURPOSE,INYECTION_PSI,PLANT_PSI,INJECTION_VOL,INJECTION_API,RETURN_VOL,RETURN_BSW,RETURN_API,VOL_REAL
0,COMP,SCH-002BTI,2015-01-02,5.0,110.0,0.0,2.0,27.4,1.786,0.0,...,Realizar prueba de 12 horas.,A,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0
1,COMP,SCH-002BTI,2015-01-05,5.0,145.0,0.0,3.0,27.4,2.027,0.0,...,None,A,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0
2,COMP,SCH-002BTI,2015-01-15,5.0,145.0,0.0,3.0,27.4,2.027,0.0,...,None,A,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0
3,COMP,SCH-002BTI,2015-01-20,5.0,122.0,0.0,3.0,27.4,2.400,0.0,...,None,A,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0
4,COMP,SCH-002BTI,2015-01-25,5.0,120.0,0.0,2.0,27.4,1.639,0.0,...,None,A,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0


In [157]:
df_production_test.columns

Index(['PTYPE', 'COMPLETION_LEGAL_NAME', 'TEST_DATE', 'T_HRS', 'TEST_OIL_24',
       'TEST_GAS_24', 'TEST_WAT_24', 'GRAV_OIL API', 'BSW', 'CASING_PRESS',
       'PBHP', 'TUBING_PRESS', 'PIP', 'FLAP', 'SBHP', 'GAS_SP_GRAVITIY',
       'TUBING_TMP', 'PI', 'AMPS_A', 'PUMP_TMP', 'MOTOR_HZ', 'POWER_KW',
       'STAGE_COUNT', 'VOLTAGE', 'INTAKE_DEPTH', 'PUMP_TYPE', 'SALINITY',
       'REMARKS', 'TEST_PURPOSE', 'INYECTION_PSI', 'PLANT_PSI',
       'INJECTION_VOL', 'INJECTION_API', 'RETURN_VOL', 'RETURN_BSW',
       'RETURN_API', 'VOL_REAL'],
      dtype='object')

In [158]:

# TODO: check the columns we need and their names

# select the columns we need
df_production_test = df_production_test[['COMPLETION_LEGAL_NAME', 'TEST_DATE', 'TEST_OIL_24', 'TEST_WAT_24', 'BSW', 'T_HRS',
                    'TEST_GAS_24', 'CASING_PRESS', 'INTAKE_DEPTH', 'MOTOR_HZ', 'PIP', 'AMPS_A', 'PUMP_TYPE', 'GRAV_OIL API']]

# rename columns
df_production_test.rename(columns={'COMPLETION_LEGAL_NAME': 'UNIQUEID', 'TEST_DATE': 'DATE',
                        'TEST_OIL_24': 'OIL_FLOW', 'TEST_WAT_24': 'WATER_FLOW', 'BSW': 'BSW', 'T_HRS': 'DURATION',
                        'TEST_GAS_24': 'GAS_FLOW', 'CASING_PRESS': 'CHP', 'INTAKE_DEPTH': 'PUMP_DEPTH', 'MOTOR_HZ': 'Frecuency',
                        'PIP': 'PIP', 'AMPS_A': 'Amperage', 'ALS': 'PUMP_TYPE', 'GRAV_OIL API': 'OIL_GRAVITY'}, inplace=True)

df_production_test.head()

,UNIQUEID,DATE,OIL_FLOW,WATER_FLOW,BSW,DURATION,GAS_FLOW,CHP,PUMP_DEPTH,Frecuency,PIP,Amperage,PUMP_TYPE,OIL_GRAVITY
0,SCH-002BTI,2015-01-02,110.0,2.0,1.786,5.0,0.0,0.0,NaN,0.0,0.0,NaN,P23 SSD,27.4
1,SCH-002BTI,2015-01-05,145.0,3.0,2.027,5.0,0.0,0.0,NaN,0.0,0.0,NaN,P23 SSD,27.4
2,SCH-002BTI,2015-01-15,145.0,3.0,2.027,5.0,0.0,0.0,NaN,0.0,0.0,NaN,P23 SSD,27.4
3,SCH-002BTI,2015-01-20,122.0,3.0,2.400,5.0,0.0,0.0,NaN,0.0,0.0,NaN,P23 SSD,27.4
4,SCH-002BTI,2015-01-25,120.0,2.0,1.639,5.0,0.0,0.0,NaN,0.0,0.0,NaN,P23 SSD,27.4


In [159]:
df_production_test['UNIQUEID'].nunique()

258

# SURVEYS

In [160]:
#  Define a function to read the survey data
def read_survey(file):
    # Initialize a variable to store the well name
    well_name = None
    
    # Open the file and extract the well name
    with open(file, 'r') as f:
        for line in f:
            if "WELL NAME:" in line:
                well_name = line.split("WELL NAME:")[1].strip()
                break

    # Initialize a variable to count header rows
    header_rows = 0

    # Open the file and determine how many header rows there are
    with open(file, 'r') as f:
        for line in f:
            if line.startswith("#"):
                header_rows += 1
            else:
                break

    # Read the data from the file, skipping the header rows
    data = pd.read_csv(file, sep='\s+', skiprows=header_rows)

    # Add a 'well_name' column with the extracted well name
    data['well_name'] = well_name

    return data.iloc[1:]  # Return the data without the first 2 rows


In [161]:
# Define the path to the directory containing survey information
path_surveys = r'C:\Users\brand\OneDrive - Universidad Central del Ecuador\Documentos\Maestría EOR\Tesis EOR\Info EPPEC\4. Información Surveys'

# List the files in the directory
files = os.listdir(path_surveys)

print(files)

['S-1.las', 'S-10.las', 'S-100.las', 'S-101.las', 'S-102.las', 'S-103.las', 'S-104.las', 'S-105.las', 'S-106.las', 'S-107.las', 'S-108.las', 'S-109.las', 'S-11.las', 'S-110.las', 'S-111.las', 'S-112.las', 'S-113.las', 'S-114.las', 'S-115.las', 'S-116.las', 'S-117.las', 'S-118.las', 'S-119.las', 'S-12.las', 'S-120.las', 'S-121.las', 'S-122.las', 'S-123.las', 'S-124.las', 'S-125.las', 'S-126.las', 'S-127.las', 'S-128.las', 'S-129.las', 'S-13.las', 'S-130.las', 'S-131.las', 'S-132.las', 'S-133.las', 'S-134.las', 'S-135.las', 'S-136.las', 'S-137.las', 'S-138.las', 'S-139.las', 'S-14.las', 'S-140D.las', 'S-141.las', 'S-142.las', 'S-143.las', 'S-144.las', 'S-145.las', 'S-146.las', 'S-147D.las', 'S-148.las', 'S-148ST.las', 'S-149.las', 'S-15.las', 'S-150D.las', 'S-151D.las', 'S-152D.las', 'S-153.las', 'S-154D.las', 'S-155D.las', 'S-156.las', 'S-157.las', 'S-158.las', 'S-159.las', 'S-16.las', 'S-160D.las', 'S-161.las', 'S-162.las', 'S-163D.las', 'S-164D.las', 'S-165D.las', 'S-166D.las', 'S-167

In [162]:
# Create an empty list to store the DataFrames
survey_data = []

# Loop through the files and read each survey data file
for file in files:
    survey_data.append(read_survey(os.path.join(path_surveys, file)))

# Concatenate the DataFrames into a single DataFrame
df_surveys = pd.concat(survey_data)

df_surveys.head()

,MD,X,Y,Z,TVD,DX,DY,AZIM,INCL,DLS,well_name
1,-0.000000000,290807.1683,9.963504e+06,883.08400,-0.0,0.0,0.0,0.0,0.0,0.0,S-1
2,10158.500000,290807.1683,9.963504e+06,-9275.41600,10158.5,0.0,0.0,0.0,0.0,0.0,S-1
1,-0.000000000,294593.0000,9.967797e+06,897.00006,-0.0,0.0,0.0,0.0,0.0,0.0,S-10
2,9964.0000000,294593.0000,9.967797e+06,-9066.99994,9964.0,0.0,0.0,0.0,0.0,0.0,S-10
1,-0.000000000,290696.9883,9.959794e+06,884.20000,-0.0,0.0,0.0,0.0,0.0,0.0,S-100


In [163]:
# change the name of the columns to Wellbore	Md	TVD	XDelt	YDelt
df_surveys.rename(columns={'well_name':'Wellbore', 'MD':'Md', 'TVD':'TVD', 'DX':'XDelt', 'DY':'YDelt'}, inplace=True)
df_surveys = df_surveys[['Wellbore', 'Md', 'TVD', 'XDelt', 'YDelt']]
df_surveys.head()

,Wellbore,Md,TVD,XDelt,YDelt
1,S-1,-0.000000000,-0.0,0.0,0.0
2,S-1,10158.500000,10158.5,0.0,0.0
1,S-10,-0.000000000,-0.0,0.0,0.0
2,S-10,9964.0000000,9964.0,0.0,0.0
1,S-100,-0.000000000,-0.0,0.0,0.0


# Datos Pozo-Capa

In [165]:
# Import the Excel files
excel_file_petro_2022 = r'C:\Users\brand\OneDrive - Universidad Central del Ecuador\Documentos\Maestría EOR\Tesis EOR\Info EPPEC\NUEVA INFORMACIÓN 06112023\Parámetros Petrofísicos pozos Sacha 2022.xlsx'
excel_file_petro_2020 = r'C:\Users\brand\OneDrive - Universidad Central del Ecuador\Documentos\Maestría EOR\Tesis EOR\Info EPPEC\NUEVA INFORMACIÓN 06112023\Parámetros Petrofísicos Sacha Dec 2020.xlsx'

# Read the Excel file into a DataFrame
df_petrophysical_2022 = pd.read_excel(excel_file_petro_2022, skiprows=1)
df_petrophysical_2020 = pd.read_excel(excel_file_petro_2020)

In [166]:
# Function to filter information
def petrophysical_processing(df_petrophysical: pd.DataFrame) -> pd.DataFrame:

    # Filter desired data
    df_petrophysical.drop(df_petrophysical[df_petrophysical['Flag Name'] == 'ROCK'].index, inplace=True)
    df_petrophysical.drop(df_petrophysical[(df_petrophysical['Zones'] != 'TLUS') & (df_petrophysical['Zones'] != 'TmTS')].index, inplace=True)
    df_petrophysical.head()

    # rename 'Zones' where injected water
    df_petrophysical['Zones'] = df_petrophysical['Zones'].replace({'TLUS': 'UI', 'TmTS': 'TI'})

    # Filter the rows with Flag Name = PAY
    df_pay = df_petrophysical.loc[df_petrophysical['Flag Name'] == 'PAY'].copy()

    # Make a filter in df_petrophysical to select rows that have the same 'Well' and 'Zones' values as df_pay
    df_res = df_petrophysical.loc[(df_petrophysical['Flag Name'] == 'RES') & (
                                    df_petrophysical['Well'].isin(df_pay['Well'])) & (
                                    df_petrophysical['Zones'].isin(df_pay['Zones']))]

    # Add to df_pay the value of Net Reservoir to df_pay
    df_pay['Gross_RES'] = df_res['Net'].values

    # Put in the desired format
    df_pay = df_pay[['Well', 'Zones', 'Gross_RES', 'Net', 'Gross', 'Av_Porosity', 'Top', 'Bottom', 'Av_PA_K']].rename(
                                    columns={'Well':'IDENTIFICADOR', 'Zones': 'CAPA', 'Net': 'ESPESOR_UTIL',
                                            'Gross_RES': 'ESPESOR_PERMEABLE', 'Gross': 'ESPESOR_TOTAL', 'Av_Porosity': 'POROSIDAD',
                                            'Top': 'TOPE_DE_CAPA', 'Bottom': 'BASE_DE_CAPA', 'Av_PA_K': 'PERMEABILIDAD_EN_X'})
    
    return df_pay

In [167]:
# Preprocessing the data in order to have the desired format and data
df_pay_2022 = petrophysical_processing(df_petrophysical_2022)
df_pay_2020 = petrophysical_processing(df_petrophysical_2020)

In [168]:
# Perform an inner merge on df_pay_2022 and df_pay_2020 to find matching rows
df_resultado = pd.merge(df_pay_2022, df_pay_2020[['IDENTIFICADOR', 'CAPA']], how='inner', on=['IDENTIFICADOR', 'CAPA'])

# Perform a left merge on df_pay_2020 and df_resultado and add an indicator column
df_pay_2020 = pd.merge(df_pay_2020, df_resultado[['IDENTIFICADOR', 'CAPA']], how='left', on=['IDENTIFICADOR', 'CAPA'], indicator=True)

# Filter out the rows where the indicator column is 'both'
df_pay_2020 = df_pay_2020[df_pay_2020['_merge'] != 'both']

# Drop the indicator column
df_pay_2020 = df_pay_2020.drop(columns=['_merge'])

# Concatenate df_pay_2022 and df_pay_2020
df_petrophysical = pd.concat([df_pay_2022, df_pay_2020], ignore_index=True, sort=False)

df_petrophysical.head()

,IDENTIFICADOR,CAPA,ESPESOR_PERMEABLE,ESPESOR_UTIL,ESPESOR_TOTAL,POROSIDAD,TOPE_DE_CAPA,BASE_DE_CAPA,PERMEABILIDAD_EN_X
0,SCHW-268R1,UI,23.054,23.054,52.942,0.149,9823.044,9875.986,464.303
1,SCHW-268R1,TI,30.570,29.568,109.500,0.127,10019.000,10128.500,260.500
2,SCHAI-415,UI,27.047,26.546,30.421,0.134,9793.579,9824.000,266.673
3,SCHAI-415,TI,24.042,24.042,60.062,0.146,10014.940,10075.000,440.430
4,SCHAI-413S1,UI,26.960,26.960,43.888,0.112,9786.162,9830.050,135.450


In [169]:
# Merge to change the name of the wells
df_petrophysical = pd.merge(df_petrophysical, df_wells_merge[['WELL_LEGAL_NAME', 'NOMBRE_COMPLETO']],
                    left_on='IDENTIFICADOR', right_on='WELL_LEGAL_NAME', how = 'left')

# Convert the NaN values in NOMBRE_COMPLETO to the values in IDENTIFICADOR
df_petrophysical['NOMBRE_COMPLETO'].fillna(df_petrophysical['IDENTIFICADOR'], inplace=True)

df_petrophysical = df_petrophysical[['NOMBRE_COMPLETO', 'CAPA', 'ESPESOR_UTIL', 'ESPESOR_PERMEABLE', 'ESPESOR_TOTAL', 'POROSIDAD', 'TOPE_DE_CAPA', 'BASE_DE_CAPA', 'PERMEABILIDAD_EN_X']].rename(
                                    columns={'NOMBRE_COMPLETO': 'IDENTIFICADOR'})

df_petrophysical.head()

,IDENTIFICADOR,CAPA,ESPESOR_UTIL,ESPESOR_PERMEABLE,ESPESOR_TOTAL,POROSIDAD,TOPE_DE_CAPA,BASE_DE_CAPA,PERMEABILIDAD_EN_X
0,S-W268R1,UI,23.054,23.054,52.942,0.149,9823.044,9875.986,464.303
1,S-W268R1,TI,29.568,30.570,109.500,0.127,10019.000,10128.500,260.500
2,S-AI415,UI,26.546,27.047,30.421,0.134,9793.579,9824.000,266.673
3,S-AI415,TI,24.042,24.042,60.062,0.146,10014.940,10075.000,440.430
4,S-AI413S1,UI,26.960,26.960,43.888,0.112,9786.162,9830.050,135.450


# CAPAS

In [170]:
# The scope of this project is Ti and Ui layers
df_capas = pd.DataFrame({'Formación': ['TI', 'UI', 'TY', 'H'],
                        'Capa': ['TI', 'UI', 'TY', 'H']
                        })

df_capas

,Formación,Capa
0,TI,TI
1,UI,UI
2,TY,TY
3,H,H


# CONVERSIONES

In [171]:
# Generate a copy of the dfs
df_mensual_Ui_Ti_temp = df_mensual_Ui_Ti.copy()
df_iny_month_temp = df_iny_month.copy()

# Add the column 'ESTADO' of each dataframe
df_mensual_Ui_Ti_temp = df_mensual_Ui_Ti_temp.assign(ESTADO='Productor')
df_iny_month_temp = df_iny_month_temp.assign(ESTADO='Inyector')

# Put in the desired format and append the dataframes
df_conversions = df_mensual_Ui_Ti_temp[['UNIQUEID', 'Date', 'ESTADO']].append(df_iny_month_temp[['UNIQUEID', 'Date', 'ESTADO']], ignore_index=True)
df_conversions.head()

C:\Users\brand\AppData\Local\Temp\ipykernel_19912\3185274696.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_conversions = df_mensual_Ui_Ti_temp[['UNIQUEID', 'Date', 'ESTADO']].append(df_iny_month_temp[['UNIQUEID', 'Date', 'ESTADO']], ignore_index=True)


,UNIQUEID,Date,ESTADO
0,SCH-002BTI,2007-11-30,Productor
1,SCH-002BTI,2007-12-31,Productor
2,SCH-002BTI,2008-01-31,Productor
3,SCH-002BTI,2008-02-29,Productor
4,SCH-002BTI,2008-03-31,Productor


In [172]:
Inject_names = df_iny_month_temp['UNIQUEID'].unique()

# with regex extract the well from compleation name
initials = [re.match(r"\D+\d+", codigo)[0] for codigo in Inject_names]

# Create a dictionary with the initials as keys and the wells as values
dict_wells = {inicial: df_conversions[df_conversions['UNIQUEID'].str.startswith(inicial)]['UNIQUEID'].unique() for inicial in initials}

for key, value in dict_wells.items():
    print(key, value)

SCH-005 ['SCH-005TI' 'SCH-005UI' 'SCH-005IT' 'SCH-005IU']
SCH-015 ['SCH-015UI' 'SCH-015IU']
SCH-023 ['SCH-023TI' 'SCH-023UI' 'SCH-023ITY']
SCH-029 ['SCH-029TI' 'SCH-029UI' 'SCH-029ITY']
SCH-036 ['SCH-036UI' 'SCH-036US' 'SCH-036IUI']
SCH-045 ['SCH-045BTI' 'SCH-045BIUI']
SCH-048 ['SCH-048IT' 'SCH-048IU']
SCH-076 ['SCH-076IT' 'SCH-076IU']
SCH-084 ['SCH-084TI' 'SCH-084UI' 'SCH-084ITY']
SCH-086 ['SCH-086UI' 'SCH-086ITY']
SCH-090 ['SCH-090U' 'SCH-090UI' 'SCH-090IU']
SCH-103 ['SCH-103UI' 'SCH-103IUI']
SCH-105 ['SCH-105IT' 'SCH-105IU']
SCH-117 ['SCH-117UI' 'SCH-117ITY']
SCHAB-315 ['SCHAB-315UI' 'SCHAB-315IUI']
SCHAD-361 ['SCHAD-361UI' 'SCHAD-361IUI']
SCHAD-363 ['SCHAD-363S1UI' 'SCHAD-363S1IUI']
SCHAE-377 ['SCHAE-377UI' 'SCHAE-377IUI']
SCHAF-382 ['SCHAF-382UI' 'SCHAF-382IUI']
SCHAG-396 ['SCHAG-396IUI']
SCHAK-431 ['SCHAK-431UI' 'SCHAK-431IUI']
SCHB-221 ['SCHB-221HTY']
SCHE-232 ['SCHE-232TI' 'SCHE-232ITI']
SCHI-067 ['SCHI-067BUI' 'SCHI-067UI' 'SCHI-067BIUI']
SCHI-200 ['SCHI-200IUI']
SCHI-201 ['SC

In [173]:
# Extract the values of the dictionary
list_values_dict = list(np.concatenate(list(dict_wells.values())))

# Filter the dataframe with the values of the dictionary
df_filtrado = df_mensual_Ui_Ti_temp[(df_mensual_Ui_Ti_temp['UNIQUEID'].isin(list_values_dict)) & (
                                    df_mensual_Ui_Ti_temp['OILP'] != 0) & (df_mensual_Ui_Ti_temp['OILP'].notna())]

# Obtaining the last month of production for each well
resultado = df_filtrado.groupby('UNIQUEID')['Date'].idxmax()

# Select the rows corresponding to the last month of each well
df_conversions_prod = df_mensual_Ui_Ti_temp.loc[resultado].reset_index(drop=True)

# Merge with df_sc to obtain the reservoir name
df_conversions_prod = pd.merge(df_conversions_prod[['UNIQUEID',	'Date', 'ESTADO']], df_sc[['COMPLETION_LEGAL_NAME', 'RESERVORIO']],
                    left_on='UNIQUEID', right_on='COMPLETION_LEGAL_NAME', how = 'left')

df_conversions_prod.rename(columns={'RESERVORIO': 'CAPA'}, inplace=True)

# Imprimimos la tabla
df_conversions_prod.head()

,UNIQUEID,Date,ESTADO,COMPLETION_LEGAL_NAME,CAPA
0,SCH-005TI,1986-09-30,Productor,SCH-005TI,TI
1,SCH-005UI,1986-09-30,Productor,SCH-005UI,UI
2,SCH-015UI,1980-12-31,Productor,SCH-015UI,UI
3,SCH-023TI,2009-10-31,Productor,SCH-023TI,TI
4,SCH-023UI,2007-12-31,Productor,SCH-023UI,UI


In [174]:
# Filter the dataframe with the values of the dictionary
df_filtrado = df_iny_month_temp[(df_iny_month_temp['UNIQUEID'].isin(list_values_dict)) & (df_iny_month_temp['WATER_INJ_VOL'] != 0) & (
                                df_iny_month_temp['WATER_INJ_VOL'].notna())]

# Obtaining the last month of injection for each well
resultado = df_filtrado.groupby('UNIQUEID')['Date'].idxmin()

# Select the rows corresponding to the last month of each well
df_conversions_iny = df_iny_month_temp.loc[resultado].reset_index(drop=True)

df_conversions_iny.head()

,UNIQUEID,CAPA,Date,INJ_TYPE,WATER_INJ_VOL,DAYS,WHP,WATER_INJ_.CUM,ESTADO
0,SCH-005IT,TI,2009-11-01,water,128971.0,30.0,NaN,128971.0,Inyector
1,SCH-005IU,UI,2009-11-01,water,49518.0,30.0,NaN,49518.0,Inyector
2,SCH-015IU,UI,2009-11-01,water,52439.0,30.0,NaN,52439.0,Inyector
3,SCH-023ITY,TY,2010-06-01,water,32451.0,30.0,NaN,32451.0,Inyector
4,SCH-029ITY,TY,2009-12-01,water,46953.0,31.0,NaN,46953.0,Inyector


In [175]:
# Append the DataFrames
df_conversions = df_conversions_prod[['UNIQUEID', 'CAPA', 'Date', 'ESTADO']].append(df_conversions_iny[['UNIQUEID', 'CAPA', 'Date', 'ESTADO']], ignore_index=True)

# Merge with df_sc to obtain the WELLBORE_ID
df_conversions = pd.merge(df_conversions, df_maestra[['COMPLETION_LEGAL_NAME', 'WELL_LEGAL_NAME', 'WELLBORE_ID']],
                    left_on='UNIQUEID', right_on='COMPLETION_LEGAL_NAME', how = 'left')

# Put in the desired format
df_conversions = df_conversions[['WELLBORE_ID', 'UNIQUEID', 'CAPA', 'Date', 'ESTADO']].rename(columns={'WELLBORE_ID': 'IDENTIFICADOR', 'Date': 'FECHA'})
df_conversions.sort_values(by=['IDENTIFICADOR', 'FECHA'], inplace=True, ignore_index=True)

df_conversions.head()

C:\Users\brand\AppData\Local\Temp\ipykernel_19912\3914423081.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_conversions = df_conversions_prod[['UNIQUEID', 'CAPA', 'Date', 'ESTADO']].append(df_conversions_iny[['UNIQUEID', 'CAPA', 'Date', 'ESTADO']], ignore_index=True)


,IDENTIFICADOR,UNIQUEID,CAPA,FECHA,ESTADO
0,S-103,SCH-103UI,UI,2017-10-31,Productor
1,S-103,SCH-103IUI,UI,2020-01-14,Inyector
2,S-105,SCH-105IT,TI,2009-11-01,Inyector
3,S-105,SCH-105IU,UI,2009-11-01,Inyector
4,S-117,SCH-117UI,UI,1999-08-31,Productor


In [176]:
# dame quellos que tienen NAN en sus valores de IDENTIFICAO
df_conversions[df_conversions['IDENTIFICADOR'].isna()]['UNIQUEID'].unique()

array(['SCHI-201IH', 'SCHB-221HTY'], dtype=object)

In [177]:
conversion_dict = {'SCH-090IU': 'S-090', 'SCHI-201IH': 'S-I201', 'SCHB-221HTY': 'S-H221', 'SCHI-200IUI': 'S-I200',
                    'SCHAB-315UI': 'S-I315', 'SCHAD-361IUI': 'S-I361', 'SCHE-232ITI': 'S-I232', 'SCHI-067BIUI': 'S-I067'}


df_conversions['IDENTIFICADOR'] = df_conversions['UNIQUEID'].map(conversion_dict).fillna(df_conversions['IDENTIFICADOR'])

df_conversions.loc[df_conversions['UNIQUEID'] == 'SCHAB-315UI', 'CAPA'] = 'UI'

df_conversions.head()

,IDENTIFICADOR,UNIQUEID,CAPA,FECHA,ESTADO
0,S-103,SCH-103UI,UI,2017-10-31,Productor
1,S-103,SCH-103IUI,UI,2020-01-14,Inyector
2,S-105,SCH-105IT,TI,2009-11-01,Inyector
3,S-105,SCH-105IU,UI,2009-11-01,Inyector
4,S-117,SCH-117UI,UI,1999-08-31,Productor


# SAVE FILTERED DATA

In [178]:
# Save dataframes to Excel file
with pd.ExcelWriter('Data_draft.xlsx') as writer:
    df_wells.to_excel(writer, sheet_name='Pozos', index=False)
    df_surveys.to_excel(writer, sheet_name='Trayectorias', index=False)
    df_perf.to_excel(writer, sheet_name='Perforados', index=False)
    df_eventos.to_excel(writer, sheet_name='Notas', index=False)
    df_mensual_Ui_Ti.to_excel(writer, sheet_name='Producción Mensual', index=False)
    df_iny_month.to_excel(writer, sheet_name='Inyección Mensual OFM', index=False)
    df_iny_month_OFM.to_excel(writer, sheet_name='Inyección Mensual Formato OFM', index=False)
    df_production_test.to_excel(writer, sheet_name='Pruebas de Producción', index=False)
    df_iny_diaria.to_excel(writer, sheet_name='Inyección Diaria OFM', index=False)
    df_capas.to_excel(writer, sheet_name='Capas', index=False)
    df_petrophysical.to_excel(writer, sheet_name='Datos Pozo-Capa', index=False)
    df_layer_status.to_excel(writer, sheet_name='Estado de Capa', index=False)
    df_conversions.to_excel(writer, sheet_name='Conversiones', index=False)
    df_notas_2.to_excel(writer, sheet_name='Completaciones Activas', index=False)

# ACUMULADOS

In [179]:
# amount of unique wells
unique_wells = list(merged_df['WELLBORE_ID'].unique())

In [180]:
def Accum_by_sand(sand):
    # Crear una lista para almacenar los resultados
    results = []

    # Iterar a través de los nombres de pozos en unique_wells
    for well_name in unique_wells:
        # Filtrar el DataFrame por 'RSVR_NAME' igual a 'Lower T Sandstone'
        filtered_df = merged_df[(merged_df['WELLBORE_ID'] == well_name) & (merged_df['RSVR_NAME'] == sand)]
        
        # Calcular la suma de 'OILP' para el pozo actual
        accum_oil = filtered_df['OILP'].sum()
        
        # Agregar el resultado a la lista de resultados
        results.append([well_name, accum_oil])

    # Crear un DataFrame a partir de la lista de resultados
    result_df = pd.DataFrame(results, columns=['Well_name', 'Accum_oil'])

    return result_df

In [181]:
Accum_T_Low = Accum_by_sand('LOWER T')
Accum_T_Low.head()

,Well_name,Accum_oil
0,S-2B,610218.002
1,S-2,0.000
2,S-3,2380627.342
3,S-5,1682342.913
4,S-6,78869.256


In [182]:
Accum_U_Low = Accum_by_sand('LOWER U')
Accum_U_Low.head()

,Well_name,Accum_oil
0,S-2B,231683.190
1,S-2,237197.000
2,S-3,99501.658
3,S-5,2626937.087
4,S-6,4433836.998
